In [3]:
from tensorflow.keras.models import Model, load_model
from tensorflow.keras.layers import Input, LSTM, Bidirectional, Dense, TimeDistributed, RepeatVector, Dropout, Concatenate
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import EarlyStopping, ReduceLROnPlateau
from tensorflow.keras.regularizers import l2
import numpy as np
import pandas as pd
import os
import joblib
import matplotlib.pyplot as plt
from sklearn.preprocessing import MinMaxScaler
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score

# [1] Define Helper Functions
def create_sequences(data, targets, window_size, forecast_horizon):
    X, y = [], []
    for i in range(len(data) - window_size - forecast_horizon):
        X.append(data[i : i + window_size])
        y.append(targets[i + window_size : i + window_size + forecast_horizon])
    return np.array(X), np.array(y)

def mean_absolute_percentage_error(y_true, y_pred): 
    return np.mean(np.abs((y_true - y_pred) / y_true)) * 100

# [2] Load and Preprocess Data
files = ['./dataset/MA-A.xlsx', './dataset/NORTHBOUND.xlsx', './dataset/SOUTHBOUND.xlsx']
sheets = ["2018", "2019", "2020", "2021", "2022", "2023"]

df_list = []
for file in files:
    if os.path.exists(file):
        for sheet in sheets:
            temp_df = pd.read_excel(file, sheet_name=sheet)

            if 'TRAFFIC STATUS' in temp_df.columns:
                temp_df.drop(columns=['TRAFFIC STATUS'], inplace=True)

            temp_df['TIME(24 HOUR)'] = temp_df['TIME(24 HOUR)'].astype(str).str.zfill(5) + ':00'
            temp_df['Datetime'] = pd.to_datetime(temp_df['DATE'].astype(str) + ' ' + temp_df['TIME(24 HOUR)'], dayfirst=True, errors='coerce')

            temp_df.dropna(subset=['Datetime'], inplace=True)
            temp_df.sort_values('Datetime', inplace=True)

            temp_df['Hour'] = temp_df['Datetime'].dt.hour
            temp_df['DayOfWeek'] = temp_df['Datetime'].dt.dayofweek
            temp_df['Month'] = temp_df['Datetime'].dt.month

            df_list.append(temp_df)

df = pd.concat(df_list, ignore_index=True)

# [3] Encode Categorical Features
categorical_cols = ['DAY OF THE WEEK', 'WEATHER', 'ROAD CONDITION', 'HOLIDAY']
df = pd.get_dummies(df, columns=categorical_cols, drop_first=True)

# Define features and target
target_column = 'TRAFFIC VOLUME'
features = df.drop([target_column, 'DATE', 'TIME(24 HOUR)', 'Datetime'], axis=1)
target = df[target_column]

# [4] Normalize Data
scaler_X = MinMaxScaler()
X_scaled = scaler_X.fit_transform(features)

scaler_y = MinMaxScaler()
y_scaled = scaler_y.fit_transform(target.values.reshape(-1, 1)).flatten()

# [5] Create Sequences
window_size = 24  # Use past 24 hours
forecast_horizon = 24  # Fixed forecast horizon for training

X_seq, y_seq = create_sequences(X_scaled, y_scaled, window_size, forecast_horizon)

# Train-Test Split
train_size = int(len(X_seq) * 0.8)
X_train, X_test = X_seq[:train_size], X_seq[train_size:]
y_train, y_test = y_seq[:train_size], y_seq[train_size:]

# [6] Build LSTM Seq2Seq Model
encoder_inputs = Input(shape=(window_size, X_train.shape[2]))

encoder_lstm1 = Bidirectional(LSTM(64, return_sequences=True, dropout=0.2, recurrent_dropout=0.2, kernel_regularizer=l2(0.01)))(encoder_inputs)
encoder_lstm2 = Bidirectional(LSTM(32, return_sequences=False, return_state=True, dropout=0.2, recurrent_dropout=0.2))

encoder_outputs, forward_h, forward_c, backward_h, backward_c = encoder_lstm2(encoder_lstm1)

state_h = Concatenate()([forward_h, backward_h])
state_c = Concatenate()([forward_c, backward_c])
encoder_states = [state_h, state_c]

decoder_inputs = RepeatVector(forecast_horizon)(state_h)

decoder_lstm1 = LSTM(64, return_sequences=True, dropout=0.2, recurrent_dropout=0.2, kernel_regularizer=l2(0.01))
decoder_lstm2 = LSTM(32, return_sequences=True, dropout=0.2, recurrent_dropout=0.2)

decoder_outputs = decoder_lstm1(decoder_inputs, initial_state=encoder_states)
decoder_outputs = decoder_lstm2(decoder_outputs)

output_layer = TimeDistributed(Dense(1, activation="linear"))(decoder_outputs)

model = Model(inputs=encoder_inputs, outputs=output_layer)

model.compile(optimizer=Adam(learning_rate=0.001), metrics=["mae"])

# [7] Train Model
early_stop = EarlyStopping(monitor="val_loss", patience=10, restore_best_weights=True)

history = model.fit(
    X_train, y_train,
    epochs=100,
    batch_size=128,  # Increase batch size
    validation_split=0.2,
    callbacks=[early_stop],
    verbose=1
)

# [8] Save Model & Scalers
model.save("traffic_forecasting_seq2seq.h5")
joblib.dump(scaler_X, "scaler_X.pkl")
joblib.dump(scaler_y, "scaler_y.pkl")

# [9] Forecasting Function (Supports 24 Hrs, Weeks, Months, Years)
def forecast_traffic(model, initial_input, forecast_steps, scaler_X, scaler_y):
    predictions = []
    input_seq = np.array(initial_input)

    for _ in range(forecast_steps):
        pred = model.predict(input_seq.reshape(1, input_seq.shape[0], input_seq.shape[1]))
        next_pred = pred[0][-1]
        predictions.append(next_pred)

        next_pred_scaled = scaler_y.transform(np.array(next_pred).reshape(-1, 1)).flatten()
        new_input = np.append(input_seq[1:], [next_pred_scaled], axis=0)
        input_seq = new_input

    predictions_real = scaler_y.inverse_transform(np.array(predictions).reshape(-1, 1)).flatten()
    return predictions_real

# [10] Forecast Next 24 Hrs, Month, and Year
last_24_hours = X_scaled[-24:]

next_24_hrs = forecast_traffic(model, last_24_hours, 24, scaler_X, scaler_y)
next_week = forecast_traffic(model, last_24_hours, 7 * 24, scaler_X, scaler_y)
next_month = forecast_traffic(model, last_24_hours, 30 * 24, scaler_X, scaler_y)


print("Next 24 Hours Forecast:", next_24_hrs)
print("Next 1 Week Forecast:", next_week)
print("Next 1 Month Forecast:", next_month)

# [11] Plot Training Loss
plt.figure(figsize=(10, 5))
plt.plot(history.history['loss'], label="Training Loss")
plt.plot(history.history['val_loss'], label="Validation Loss")
plt.title("Training vs Validation Loss")
plt.xlabel("Epochs")
plt.ylabel("Loss (MSE)")
plt.legend()
plt.show()

Epoch 1/100


/home/jzier/null/work/Traffic-Forecasting/traffic_forecasting/lib/python3.12/site-packages/keras/src/optimizers/base_optimizer.py:774: UserWarning: Gradients do not exist for variables ['bidirectional_4/forward_lstm_8/lstm_cell/recurrent_kernel', 'bidirectional_4/forward_lstm_8/lstm_cell/bias', 'bidirectional_4/backward_lstm_8/lstm_cell/recurrent_kernel', 'bidirectional_4/backward_lstm_8/lstm_cell/bias', 'bidirectional_5/forward_lstm_9/lstm_cell/kernel', 'bidirectional_5/forward_lstm_9/lstm_cell/recurrent_kernel', 'bidirectional_5/forward_lstm_9/lstm_cell/bias', 'bidirectional_5/backward_lstm_9/lstm_cell/kernel', 'bidirectional_5/backward_lstm_9/lstm_cell/recurrent_kernel', 'bidirectional_5/backward_lstm_9/lstm_cell/bias', 'lstm_10/lstm_cell/recurrent_kernel', 'lstm_10/lstm_cell/bias', 'lstm_11/lstm_cell/kernel', 'lstm_11/lstm_cell/recurrent_kernel', 'lstm_11/lstm_cell/bias', 'time_distributed_2/dense_2/kernel', 'time_distributed_2/dense_2/bias'] when minimizing the loss. If using `mod

311/311 ━━━━━━━━━━━━━━━━━━━━ 9s 21ms/step - loss: 0.5961 - mae: 0.1611 - val_loss: 4.1045e-05 - val_mae: 0.1853
Epoch 2/100
311/311 ━━━━━━━━━━━━━━━━━━━━ 5s 16ms/step - loss: 9.0453e-06 - mae: 0.1528 - val_loss: 1.8163e-10 - val_mae: 0.1852
Epoch 3/100
311/311 ━━━━━━━━━━━━━━━━━━━━ 5s 16ms/step - loss: 1.4145e-10 - mae: 0.1515 - val_loss: 1.0831e-10 - val_mae: 0.1852
Epoch 4/100
311/311 ━━━━━━━━━━━━━━━━━━━━ 5s 16ms/step - loss: 6.6544e-10 - mae: 0.1535 - val_loss: 1.0811e-09 - val_mae: 0.1852
Epoch 5/100
311/311 ━━━━━━━━━━━━━━━━━━━━ 5s 17ms/step - loss: 6.8887e-10 - mae: 0.1526 - val_loss: 1.0424e-09 - val_mae: 0.1852
Epoch 6/100
311/311 ━━━━━━━━━━━━━━━━━━━━ 5s 16ms/step - loss: 1.1667e-09 - mae: 0.1529 - val_loss: 7.1785e-10 - val_mae: 0.1852
Epoch 7/100
311/311 ━━━━━━━━━━━━━━━━━━━━ 5s 16ms/step - loss: 1.1075e-09 - mae: 0.1520 - val_loss: 1.0433e-09 - val_mae: 0.1852
Epoch 8/100
311/311 ━━━━━━━━━━━━━━━━━━━━ 5s 16ms/step - loss: 9.5065e-10 - mae: 0.1526 - val_loss: 1.6559e-09 - val_mae:

1/1 ━━━━━━━━━━━━━━━━━━━━ 1s 959ms/step


ValueError: all the input array dimensions except for the concatenation axis must match exactly, but along dimension 1, the array at index 0 has size 18 and the array at index 1 has size 1

In [ ]:
# [9] Training Predictions
y_train_pred_scaled = model.predict(X_train)

# Inverse transform predictions
y_train_pred = scaler_y.inverse_transform(y_train_pred_scaled.reshape(-1, forecast_horizon))
y_train_actual = scaler_y.inverse_transform(y_train.reshape(-1, forecast_horizon))

# Select only the first predicted step for each sample
y_train_actual_first = y_train_actual[:, 0]  # First actual timestep
y_train_pred_first = y_train_pred[:, 0]      # First predicted timestep

# Plot actual vs predicted (Training Set)
plt.figure(figsize=(15, 5))
plt.plot(y_train_actual_first[:250], label="Actual")
plt.plot(y_train_pred_first[:250], label="Predicted")
plt.title("Actual vs Predicted Traffic Volume (Training Set - First 250 Samples)")
plt.xlabel("Samples")
plt.ylabel("Traffic Volume")
plt.legend()
plt.show()

In [6]:
import os
import numpy as np
import pandas as pd
import joblib
from tensorflow.keras.models import load_model
from tensorflow.keras.losses import MeanSquaredError

# ✅ Load Model and Scalers
try:
    model = load_model("traffic_forecasting_seq2seq.h5", custom_objects={"mse": MeanSquaredError()})
    scaler_X = joblib.load("scaler_X.pkl")
    scaler_y = joblib.load("scaler_y.pkl")
    print("📂 Model and scalers loaded successfully.")
except Exception as e:
    print(f"❌ Error loading model or scalers: {e}")
    exit()

# ✅ Get User Input
start_date = input("Enter the start date for prediction (YYYY-MM-DD): ").strip()
end_date = input("Enter the end date for prediction (YYYY-MM-DD): ").strip()

try:
    start_datetime = pd.to_datetime(start_date, format='%Y-%m-%d')
    end_datetime = pd.to_datetime(end_date, format='%Y-%m-%d')
except Exception:
    print("⚠️ Invalid date format! Use YYYY-MM-DD.")
    exit()

if start_datetime == end_datetime:
    end_datetime += pd.Timedelta(days=1)

forecast_horizon = int((end_datetime - start_datetime).total_seconds() // 3600)
if forecast_horizon < 1:
    print("⚠️ The selected date range must include at least one hour for prediction.")
    exit()

# ✅ Load Dataset
dataset_path = "./dataset/NORTHBOUND.xlsx"
if not os.path.exists(dataset_path):
    print(f"⚠️ Dataset file '{dataset_path}' not found!")
    exit()

try:
    df = pd.read_excel(dataset_path)
except Exception as e:
    print(f"⚠️ Error loading dataset: {e}")
    exit()

# ✅ Fix Timestamp Parsing
try:
    df["DATE"] = pd.to_datetime(df["DATE"].astype(str).str.strip(), errors="coerce")

    # ✅ Fix "TIME(24 HOUR)" Formatting
    df["TIME(24 HOUR)"] = df["TIME(24 HOUR)"].astype(str).str.strip()  # Remove spaces
    df["TIME(24 HOUR)"] = df["TIME(24 HOUR)"].apply(lambda x: x.zfill(5))  # Ensure HH:MM format
    df["TIME(24 HOUR)"] = pd.to_datetime(df["TIME(24 HOUR)"], format="%H:%M", errors="coerce").dt.hour

    # ✅ Create "Datetime" column
    df["Datetime"] = df["DATE"] + pd.to_timedelta(df["TIME(24 HOUR)"], unit='h')
    df.sort_values("Datetime", inplace=True)

    if df["Datetime"].isnull().any():
        raise ValueError("⚠️ Some timestamps could not be parsed.")

    # ✅ Get Last Known Timestamp
    last_known_timestamp = df["Datetime"].iloc[-1]

except Exception as e:
    print(f"⚠️ Error parsing timestamp from dataset: {e}")
    exit()

# ✅ Add Missing Derived Features (Hour, DayOfWeek, Month)
df["Hour"] = df["Datetime"].dt.hour
df["DayOfWeek"] = df["Datetime"].dt.dayofweek
df["Month"] = df["Datetime"].dt.month

# ✅ One-Hot Encode Categorical Features (Same as Training)
categorical_cols = ['DAY OF THE WEEK', 'WEATHER', 'ROAD CONDITION', 'HOLIDAY']
df = pd.get_dummies(df, columns=categorical_cols, drop_first=True)

# ✅ Select Only Relevant Columns
try:
    feature_columns = scaler_X.feature_names_in_
    df = df[feature_columns]  # Keep only columns used during training
except Exception as e:
    print(f"⚠️ Feature mismatch: {e}")
    exit()

# ✅ Scale Input Data
try:
    X_scaled = scaler_X.transform(df.values)
except Exception as e:
    print(f"⚠️ Error scaling input data: {e}")
    exit()

# ✅ Ensure X_scaled Exists
if X_scaled.shape[0] < 24:
    print("⚠️ Not enough historical data (require last 24 hours).")
    exit()

# ✅ Get Last 24 Hours of Data for Prediction
latest_data = X_scaled[-24:].reshape(1, 24, X_scaled.shape[1])  # (1, 24, features)

# ✅ Iterative Multi-Step Forecasting
y_pred_scaled_list = []
timestamps = []
# ✅ Adjust Timestamp Handling to Full Hours
current_timestamp = start_datetime.replace(minute=0, second=0, microsecond=0)  # Start at the top of the hour

for i in range(forecast_horizon):
    y_pred_scaled = model.predict(latest_data)  # Output shape: (1, 24, 1)

    print(f"🔍 Prediction {i+1}/{forecast_horizon}, y_pred_scaled shape: {y_pred_scaled.shape}")

    # ✅ Extract the first step (convert from (1, 24, 1) → (1, 1, 1))
    y_pred_single = y_pred_scaled[:, 0, :].reshape(1, 1, 1)

    # ✅ Store Prediction
    y_pred_scaled_list.append(y_pred_single.flatten())
    
    # ✅ Ensure timestamp is rounded to the nearest hour
    timestamps.append(current_timestamp.strftime("%Y-%m-%d %H:%M:%S"))  # Format as "YYYY-MM-DD HH:00:00"
    
    # ✅ Update input sequence (Shift left, append new prediction)
    latest_data = np.hstack((latest_data[:, 1:, :], np.zeros((1, 1, latest_data.shape[-1]))))  # Shift left
    latest_data[:, -1, -1] = y_pred_single.flatten()  # Insert new prediction in last column

    # ✅ Update current timestamp after each iteration
    current_timestamp += pd.Timedelta(hours=1)  # Increment by 1 hour

# ✅ Convert Predictions Back to Original Scale
y_pred_scaled_array = np.array(y_pred_scaled_list).reshape(-1, 1)
y_pred_actual = scaler_y.inverse_transform(y_pred_scaled_array).flatten()

# ✅ Display Predictions
prediction_df = pd.DataFrame({"Datetime": timestamps, "Predicted Traffic Volume": y_pred_actual})
print("\n📊 Predicted Traffic Volume:")
print(prediction_df)

📂 Model and scalers loaded successfully.


Enter the start date for prediction (YYYY-MM-DD):  2025-03-18
Enter the end date for prediction (YYYY-MM-DD):  2025-03-20


/home/jzier/null/work/Traffic-Forecasting/traffic_forecasting/lib/python3.12/site-packages/sklearn/utils/validation.py:2739: UserWarning: X does not have valid feature names, but MinMaxScaler was fitted with feature names
  warnings.warn(


1/1 ━━━━━━━━━━━━━━━━━━━━ 2s 2s/step
🔍 Prediction 1/48, y_pred_scaled shape: (1, 24, 1)
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 37ms/step
🔍 Prediction 2/48, y_pred_scaled shape: (1, 24, 1)
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 38ms/step
🔍 Prediction 3/48, y_pred_scaled shape: (1, 24, 1)
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 39ms/step
🔍 Prediction 4/48, y_pred_scaled shape: (1, 24, 1)
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 38ms/step
🔍 Prediction 5/48, y_pred_scaled shape: (1, 24, 1)
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 37ms/step
🔍 Prediction 6/48, y_pred_scaled shape: (1, 24, 1)
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 38ms/step
🔍 Prediction 7/48, y_pred_scaled shape: (1, 24, 1)
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 43ms/step
🔍 Prediction 8/48, y_pred_scaled shape: (1, 24, 1)
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 44ms/step
🔍 Prediction 9/48, y_pred_scaled shape: (1, 24, 1)
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 40ms/step
🔍 Prediction 10/48, y_pred_scaled shape: (1, 24, 1)
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 40ms/step
🔍 Prediction 11/48, y_pred_scaled shape: (1, 24, 1)
1/1 ━━━━━━━━━━━━━━━━━